In [153]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [154]:
import google.generativeai as genai

In [155]:
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

In [156]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [157]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [158]:
# model = genai.GenerativeModel('gemini-pro')
# response = model.generate_content("How to stay active as a male?")
# print(response.text)


In [159]:
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [160]:
model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.8)

chain = load_qa_chain(model, chain_type='stuff', prompt=prompt)


In [161]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [162]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
context = """

Dear Arko,
I hope you're doing well! I recently visited Shimla and had an amazing time, so I wanted to
share a bit about my trip with you.
The journey started with the toy train ride from Kalka, which was an incredible experience
with its scenic views and winding tracks. Shimla itself is beautiful, with its colonial
architecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling
along Mall Road, exploring shops, and indulging in local cuisine.
A visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along
the way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried
horse riding.
The locals were warm and welcoming, adding to the charm of the trip. Shimla was the
perfect escape from the city's hustle and bustle.
I wish you could have been there to share the experience. Let's plan a trip together soon!
Take care and stay in touch!
Warm regards,
Swastik Dhar

"""
texts = text_splitter.split_text(context)

In [163]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [164]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever()


In [165]:
question = "Is Arko Swastik's friend?"
docs = vector_index.invoke(question)


In [166]:
docs

[Document(page_content="Dear Arko,\nI hope you're doing well! I recently visited Shimla and had an amazing time, so I wanted to\nshare a bit about my trip with you.\nThe journey started with the toy train ride from Kalka, which was an incredible experience\nwith its scenic views and winding tracks. Shimla itself is beautiful, with its colonial\narchitecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling\nalong Mall Road, exploring shops, and indulging in local cuisine.\nA visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along\nthe way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried\nhorse riding.\nThe locals were warm and welcoming, adding to the charm of the trip. Shimla was the\nperfect escape from the city's hustle and bustle.\nI wish you could have been there to share the experience. Let's plan a trip together soon!\nTake care and stay in touch!\nWarm regards,\nSwastik Dhar"),

In [167]:
stuff_answer = chain.invoke(
    {"input_documents": docs, "question": question}, return_only_outputs=True
)


In [170]:
stuff_answer['output_text']

'Yes'